In [1]:
import torch
import random
import torchvision
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.nn.functional as F
import argparse,os,time
import os
import time
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
num_gpus=4

In [2]:
data=pd.read_csv("./Titanic_data/train.csv")

In [3]:
data

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C


In [4]:
train_data=data.iloc[:,[1,2,4,5,6,7]]
train_data

,Survived,Pclass,Sex,Age,SibSp,Parch
0,0,3,male,22.0,1,0
1,1,1,female,38.0,1,0
2,1,3,female,26.0,0,0
3,1,1,female,35.0,1,0
4,0,3,male,35.0,0,0
...,...,...,...,...,...,...
886,0,2,male,27.0,0,0
887,1,1,female,19.0,0,0
888,0,3,female,NaN,1,2
889,1,1,male,26.0,0,0


In [5]:
train_data=train_data.replace("male",0)
train_data=train_data.replace("female",1)
train_data=train_data.dropna(axis=0)
train_data

,Survived,Pclass,Sex,Age,SibSp,Parch
0,0,3,0,22.0,1,0
1,1,1,1,38.0,1,0
2,1,3,1,26.0,0,0
3,1,1,1,35.0,1,0
4,0,3,0,35.0,0,0
...,...,...,...,...,...,...
885,0,3,1,39.0,0,5
886,0,2,0,27.0,0,0
887,1,1,1,19.0,0,0
889,1,1,0,26.0,0,0


In [6]:
y_data=train_data.loc[:,"Survived"].values
x_data=train_data.loc[:,"Pclass":].values

In [7]:
mean_x=np.mean(x_data, axis=0)
std_x=np.std(x_data, axis=0)
x_train=(x_data - mean_x) / std_x

In [8]:
class CustomDataset(Dataset):
    def __init__(self,x_dat,y_dat):
        x = x_dat
        y = y_dat
        self.len = x.shape[0]
        y=y.astype('int')
        x=x.astype('float32')
        self.x_data = torch.tensor(x)
        self.y_data = torch.tensor(y)

    def __getitem__(self, index):
        return self.x_data[index], self.y_data[index]

    def __len__(self):
        return self.len

In [9]:
train_data_x, val_data_x, train_data_label, val_data_label = train_test_split(x_data, y_data, 
                                                                        test_size=0.2)

In [10]:
class Model(torch.nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        self.fc = nn.Sequential(
            nn.Linear(5, 1000),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(1000, 1000),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(1000, 1000),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(1000, 1000),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(1000, 1000),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(1000, 1000),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(1000, 1000),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(1000, 2),
            nn.LeakyReLU(0.2, inplace=True)
        )

    def forward(self, x):
        x=x.view(batch_size//num_gpus,-1)
        out=self.fc(x)
        return out

In [11]:
batch_size=32
train_dataset = CustomDataset(train_data_x,train_data_label)
train_loader = DataLoader(dataset=train_dataset,pin_memory=True,
                          batch_size=batch_size,
                          shuffle=True,
                          num_workers=60,drop_last=True)
val_dataset = CustomDataset(val_data_x,val_data_label)
val_loader = DataLoader(dataset=val_dataset,pin_memory=True,
                          batch_size=batch_size,
                          shuffle=True,
                          num_workers=60,drop_last=True)

In [12]:
model=nn.DataParallel(Model().cuda())
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(),weight_decay=0.005)

In [13]:
trn_loss_list = []
val_loss_list = []
total_epoch=100
model_char="minloss"
model_name=""
patience=5
start_early_stop_check=0
saving_start_epoch=10

for epoch in range(total_epoch):
    trn_loss = 0.0
    for i, data in enumerate(train_loader, 0):
        inputs, labels = data
        if torch.cuda.is_available():
            inputs=inputs.cuda()
            labels=labels.cuda()
        # grad init
        optimizer.zero_grad()
        # forward propagation
        output= model(inputs)
        # calculate loss
        loss=criterion(output, labels)
        # back propagation 
        loss.backward()
        # weight update
        optimizer.step()
        
        # trn_loss summary
        trn_loss += loss.item()
        # del (memory issue)
        del loss
        del output
    with torch.no_grad():
        val_loss = 0.0
        cor_match = 0
        for j, val in enumerate(val_loader):
            val_x, val_label = val
            if torch.cuda.is_available():
                val_x = val_x.cuda()
                val_label =val_label.cuda()
            val_output = model(val_x)
            v_loss = criterion(val_output, val_label)
            val_loss += v_loss
            _, predicted=torch.max(val_output,1)
            cor_match+=np.count_nonzero(predicted.cpu().detach()==val_label.cpu().detach())
    del val_output
    del v_loss
    del predicted
    
    
    
    trn_loss_list.append(trn_loss/len(train_loader))
    val_loss_list.append(val_loss/len(val_loader))
    val_acc=cor_match/(len(val_loader)*batch_size)
    now = time.localtime()
    print ("%04d/%02d/%02d %02d:%02d:%02d" % (now.tm_year, now.tm_mon, now.tm_mday, now.tm_hour, now.tm_min, now.tm_sec))

    print("epoch: {}/{} | trn loss: {:.4f} | val loss: {:.4f} | val accuracy: {:.4f}% \n".format(
                epoch+1, total_epoch, trn_loss / len(train_loader), val_loss / len(val_loader), val_acc*100
            ))
    
    
    if epoch+1>2:
        if val_loss_list[-1]>val_loss_list[-2]:
            start_early_stop_check=1
    else:
        val_loss_min=val_loss_list[-1]
        
    if start_early_stop_check:
        early_stop_temp=val_loss_list[-patience:]
        if all(early_stop_temp[i]<early_stop_temp[i+1] for i in range (len(early_stop_temp)-1)):
            print("Early stop!")
            break
            
    if epoch+1>saving_start_epoch:
        if val_loss_list[-1]<val_loss_min:
            if os.path.isfile(model_name):
                os.remove(model_name)
            val_loss_min=val_loss_list[-1]
            model_name="Custom_model_"+model_char+"_{:.3f}".format(val_loss_min)
            torch.save(model, model_name)
            print("Model replaced and saved as ",model_name)

2020/09/28 11:38:34
epoch: 1/100 | trn loss: 0.7586 | val loss: 0.6613 | val accuracy: 62.5000% 

2020/09/28 11:38:42
epoch: 2/100 | trn loss: 0.6856 | val loss: 0.6780 | val accuracy: 67.1875% 

2020/09/28 11:38:49
epoch: 3/100 | trn loss: 0.6867 | val loss: 0.6684 | val accuracy: 65.6250% 

2020/09/28 11:38:57
epoch: 4/100 | trn loss: 0.6835 | val loss: 0.6629 | val accuracy: 65.6250% 

2020/09/28 11:39:05
epoch: 5/100 | trn loss: 0.6832 | val loss: 0.6753 | val accuracy: 64.8438% 

2020/09/28 11:39:13
epoch: 6/100 | trn loss: 0.6834 | val loss: 0.6604 | val accuracy: 64.8438% 

2020/09/28 11:39:21
epoch: 7/100 | trn loss: 0.6837 | val loss: 0.6649 | val accuracy: 65.6250% 

2020/09/28 11:39:29
epoch: 8/100 | trn loss: 0.6778 | val loss: 0.6537 | val accuracy: 63.2812% 

2020/09/28 11:39:37
epoch: 9/100 | trn loss: 0.6830 | val loss: 0.6689 | val accuracy: 65.6250% 

2020/09/28 11:39:44
epoch: 10/100 | trn loss: 0.6846 | val loss: 0.6698 | val accuracy: 64.8438% 

2020/09/28 11:39:52

/home/jupyter-kunwoopark/.local/lib/python3.7/site-packages/torch/serialization.py:402: UserWarning: Couldn't retrieve source code for container of type Model. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


2020/09/28 11:40:00
epoch: 12/100 | trn loss: 0.6799 | val loss: 0.6572 | val accuracy: 66.4062% 

Model replaced and saved as  Custom_model_minloss_0.657
2020/09/28 11:40:08
epoch: 13/100 | trn loss: 0.6789 | val loss: 0.6446 | val accuracy: 67.9688% 

Model replaced and saved as  Custom_model_minloss_0.645
2020/09/28 11:40:16
epoch: 14/100 | trn loss: 0.6815 | val loss: 0.6647 | val accuracy: 63.2812% 

2020/09/28 11:40:24
epoch: 15/100 | trn loss: 0.6814 | val loss: 0.6600 | val accuracy: 66.4062% 

2020/09/28 11:40:32
epoch: 16/100 | trn loss: 0.6796 | val loss: 0.6603 | val accuracy: 64.8438% 

2020/09/28 11:40:39
epoch: 17/100 | trn loss: 0.6791 | val loss: 0.6575 | val accuracy: 64.8438% 

2020/09/28 11:40:47
epoch: 18/100 | trn loss: 0.6813 | val loss: 0.6487 | val accuracy: 67.9688% 

2020/09/28 11:40:55
epoch: 19/100 | trn loss: 0.6818 | val loss: 0.6592 | val accuracy: 65.6250% 

2020/09/28 11:41:03
epoch: 20/100 | trn loss: 0.6788 | val loss: 0.6678 | val accuracy: 62.5000%

2020/09/28 11:50:52
epoch: 94/100 | trn loss: 0.6815 | val loss: 0.6617 | val accuracy: 64.8438% 

2020/09/28 11:51:00
epoch: 95/100 | trn loss: 0.6810 | val loss: 0.6550 | val accuracy: 66.4062% 

2020/09/28 11:51:08
epoch: 96/100 | trn loss: 0.6791 | val loss: 0.6590 | val accuracy: 64.8438% 

2020/09/28 11:51:16
epoch: 97/100 | trn loss: 0.6832 | val loss: 0.6624 | val accuracy: 64.0625% 

2020/09/28 11:51:25
epoch: 98/100 | trn loss: 0.6803 | val loss: 0.6611 | val accuracy: 64.8438% 

2020/09/28 11:51:33
epoch: 99/100 | trn loss: 0.6813 | val loss: 0.6582 | val accuracy: 65.6250% 

2020/09/28 11:51:41
epoch: 100/100 | trn loss: 0.6798 | val loss: 0.6536 | val accuracy: 66.4062% 

